# SQL Session

### Main entry point for DataFrame and SQL functionality

In [1]:
from pyspark.sql import SparkSession
# SparkSession --> entry point to programming Spark with the Dataset and DataFrame API

### Start SPARK session

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

# Builder Class ---> To create SparkSession Instances
# AppName ---> Sets a name for the application, which will be shown in the Spark web UI


### Import JSON File

In [3]:
Apply =spark.read.json('/home/bluepi/Documents/BluePi/Apply.json')
College =spark.read.json('/home/bluepi/Documents/BluePi/College.json')
Student =spark.read.json('/home/bluepi/Documents/BluePi/Student.json')

### Show the Data

In [4]:
Apply.show()

+--------+--------+--------------+---+
|   cName|decision|         major|sID|
+--------+--------+--------------+---+
|Stanford|       Y|            CS|123|
|Stanford|       N|            EE|123|
|Berkeley|       Y|            CS|123|
| Cornell|       Y|            EE|123|
|Berkeley|       N|       biology|234|
|     MIT|       Y|bioengineering|345|
| Cornell|       N|bioengineering|345|
| Cornell|       Y|            CS|345|
| Cornell|       N|            EE|345|
|Stanford|       Y|       history|678|
|Stanford|       Y|            CS|987|
|Berkeley|       Y|            CS|987|
|Stanford|       N|            CS|876|
|     MIT|       Y|       biology|876|
|     MIT|       N|marine biology|876|
|Stanford|       Y|       history|765|
| Cornell|       N|       history|765|
| Cornell|       Y|    psychology|765|
|     MIT|       N|            CS|543|
+--------+--------+--------------+---+



In [5]:
College.show()

+--------+----------+-----+
|   cName|enrollment|state|
+--------+----------+-----+
|Stanford|     15000|   CA|
|Berkeley|     36000|   CA|
|     MIT|     10000|   MA|
| Cornell|     21000|   NY|
| Harvard|     50040|   MA|
+--------+----------+-----+



In [6]:
Student.show()

+---+---+------+------+
|GPA|sID| sName|sizeHS|
+---+---+------+------+
|3.9|123|   Amy|  1000|
|3.6|234|   Bob|  1500|
|3.5|345| Craig|   500|
|3.9|456| Doris|  1000|
|2.9|567|Edward|  2000|
|3.8|678|   Fay|   200|
|3.4|789|  Gary|   800|
|3.7|987| Helen|   800|
|3.9|876| Irene|   400|
|2.9|765|   Jay|  1500|
|3.9|654|   Amy|  1000|
|3.4|543| Craig|  2000|
+---+---+------+------+



### PRINT Schema of DataFrame

In [7]:

Apply.printSchema()

root
 |-- cName: string (nullable = true)
 |-- decision: string (nullable = true)
 |-- major: string (nullable = true)
 |-- sID: long (nullable = true)



In [8]:
College.printSchema()

root
 |-- cName: string (nullable = true)
 |-- enrollment: long (nullable = true)
 |-- state: string (nullable = true)



In [9]:
Student.printSchema()

root
 |-- GPA: double (nullable = true)
 |-- sID: long (nullable = true)
 |-- sName: string (nullable = true)
 |-- sizeHS: long (nullable = true)



### Registering DataFrame as a SQL temp VIEW

In [10]:
Apply.createOrReplaceTempView("ApplyTable")
College.createOrReplaceTempView("CollegeTable")
Student.createOrReplaceTempView("StudentTable")

### To run SQL Queries Directly

In [11]:
result = spark.sql("SELECT * FROM StudentTable")
result.show()

+---+---+------+------+
|GPA|sID| sName|sizeHS|
+---+---+------+------+
|3.9|123|   Amy|  1000|
|3.6|234|   Bob|  1500|
|3.5|345| Craig|   500|
|3.9|456| Doris|  1000|
|2.9|567|Edward|  2000|
|3.8|678|   Fay|   200|
|3.4|789|  Gary|   800|
|3.7|987| Helen|   800|
|3.9|876| Irene|   400|
|2.9|765|   Jay|  1500|
|3.9|654|   Amy|  1000|
|3.4|543| Craig|  2000|
+---+---+------+------+



- Q1 Produce a combine table in which each student is combine with every other application.

In [12]:
spark.sql(" select S.GPA,s.sID,s.sName,s.sizeHS,a.cName,a.decision,a.major \
              from StudentTable S full outer join ApplyTable A \
                  on s.sid = a.sid ").show()

+---+---+------+------+--------+--------+--------------+
|GPA|sID| sName|sizeHS|   cName|decision|         major|
+---+---+------+------+--------+--------+--------------+
|3.9|876| Irene|   400|Stanford|       N|            CS|
|3.9|876| Irene|   400|     MIT|       Y|       biology|
|3.9|876| Irene|   400|     MIT|       N|marine biology|
|3.4|789|  Gary|   800|    null|    null|          null|
|3.6|234|   Bob|  1500|Berkeley|       N|       biology|
|3.8|678|   Fay|   200|Stanford|       Y|       history|
|3.9|654|   Amy|  1000|    null|    null|          null|
|3.9|123|   Amy|  1000|Stanford|       Y|            CS|
|3.9|123|   Amy|  1000|Stanford|       N|            EE|
|3.9|123|   Amy|  1000|Berkeley|       Y|            CS|
|3.9|123|   Amy|  1000| Cornell|       Y|            EE|
|3.5|345| Craig|   500|     MIT|       Y|bioengineering|
|3.5|345| Craig|   500| Cornell|       N|bioengineering|
|3.5|345| Craig|   500| Cornell|       Y|            CS|
|3.5|345| Craig|   500| Cornell

- Q2 Give Student ID, name, GPA and name of college and major each student applied to.

In [13]:
spark.sql(" select s.sid, s.sname, s.gpa, a.cname, a.major from studenttable s inner join applytable a \
                on s.sid = a.sid ").show()

+---+-----+---+--------+--------------+
|sid|sname|gpa|   cname|         major|
+---+-----+---+--------+--------------+
|123|  Amy|3.9|Stanford|            CS|
|123|  Amy|3.9|Stanford|            EE|
|123|  Amy|3.9|Berkeley|            CS|
|123|  Amy|3.9| Cornell|            EE|
|234|  Bob|3.6|Berkeley|       biology|
|345|Craig|3.5|     MIT|bioengineering|
|345|Craig|3.5| Cornell|bioengineering|
|345|Craig|3.5| Cornell|            CS|
|345|Craig|3.5| Cornell|            EE|
|678|  Fay|3.8|Stanford|       history|
|987|Helen|3.7|Stanford|            CS|
|987|Helen|3.7|Berkeley|            CS|
|876|Irene|3.9|Stanford|            CS|
|876|Irene|3.9|     MIT|       biology|
|876|Irene|3.9|     MIT|marine biology|
|765|  Jay|2.9|Stanford|       history|
|765|  Jay|2.9| Cornell|       history|
|765|  Jay|2.9| Cornell|    psychology|
|543|Craig|3.4|     MIT|            CS|
+---+-----+---+--------+--------------+



- Q3. Find detail of applications who applied to California State.

In [14]:
spark.sql(" select a.sid,a.cname,a.major,a.decision from ApplyTable A left join CollegeTable C \
                on a.cname = c.cname where c.state = 'CA' ").show()

+---+--------+-------+--------+
|sid|   cname|  major|decision|
+---+--------+-------+--------+
|123|Stanford|     CS|       Y|
|123|Stanford|     EE|       N|
|123|Berkeley|     CS|       Y|
|234|Berkeley|biology|       N|
|678|Stanford|history|       Y|
|987|Stanford|     CS|       Y|
|987|Berkeley|     CS|       Y|
|876|Stanford|     CS|       N|
|765|Stanford|history|       Y|
+---+--------+-------+--------+



- Q4. IDs, name, GPA of students and name of college with GPA > 3.7 applying to Stanford

In [15]:
spark.sql(" select s.sid, s.sname, s.gpa, a.cname from studenttable s inner join applytable a \
                on s.sid = a.sid where gpa > 3.7 ").show()

+---+-----+---+--------+
|sid|sname|gpa|   cname|
+---+-----+---+--------+
|123|  Amy|3.9|Stanford|
|123|  Amy|3.9|Stanford|
|123|  Amy|3.9|Berkeley|
|123|  Amy|3.9| Cornell|
|678|  Fay|3.8|Stanford|
|876|Irene|3.9|Stanford|
|876|Irene|3.9|     MIT|
|876|Irene|3.9|     MIT|
+---+-----+---+--------+



- Q5. Find detail of Student who apply to CS major and their application are rejected

In [16]:
spark.sql(" select s.sid, s.sname, s.gpa from studenttable s inner join applytable a \
                on s.sid = a.sid where a.major = 'CS' and a.decision = 'N' ").show()

+---+-----+---+
|sid|sname|gpa|
+---+-----+---+
|876|Irene|3.9|
|543|Craig|3.4|
+---+-----+---+



- Q6. Find detail of student and application who applied to colleges at New York


In [17]:
spark.sql(" select s.sid, s.sname, s.gpa, a.cname, a.major, a.decision \
                from studenttable s inner join applytable a \
                    on s.sid = a.sid \
                        inner join collegetable c \
                            on a.cname = c.cname \
                                where c.state = 'NY' ").show()

+---+-----+---+-------+--------------+--------+
|sid|sname|gpa|  cname|         major|decision|
+---+-----+---+-------+--------------+--------+
|123|  Amy|3.9|Cornell|            EE|       Y|
|345|Craig|3.5|Cornell|bioengineering|       N|
|345|Craig|3.5|Cornell|            CS|       Y|
|345|Craig|3.5|Cornell|            EE|       N|
|765|  Jay|2.9|Cornell|       history|       N|
|765|  Jay|2.9|Cornell|    psychology|       Y|
+---+-----+---+-------+--------------+--------+



- Q7. Find detail of student who have not applied to any of college


In [18]:
#spark.sql(" select s.sid, s.sname, s.gpa from studenttable s where s.sid not in ( select sid from applytable ) ").show()
spark.sql(" select s.sid, s.sname, s.gpa from studenttable s left join applytable a \
                on s.sid = a.sid where a.cname is NULL ").show()

+---+------+---+
|sid| sname|gpa|
+---+------+---+
|456| Doris|3.9|
|567|Edward|2.9|
|789|  Gary|3.4|
|654|   Amy|3.9|
+---+------+---+



- Q8. Find college where no student have applied


In [19]:
# spark.sql(" select cname from collegetable where cname not in ( select cname from applytable)").show()
spark.sql(" select c.cname from collegetable c left join applytable a on c.cname = a.cname \
                where a.sid is NULL ").show()

+-------+
|  cname|
+-------+
|Harvard|
+-------+



- Q9. Find sID who have only one application


In [20]:
spark.sql(" select a1.sid from applytable as a1 inner join \
          (select sid, count(sid) as c_sid from applytable group by sid having count(sid) = 1) as a2 \
          on a1.sid = a2.sid").show()



+---+
|sid|
+---+
|234|
|678|
|543|
+---+



- Q10. Find name and GPA of applicants who apply to any college whose enrollment is not more than 25000.


In [21]:
spark.sql(" select s.sname, s.gpa from studenttable as s \
                inner join applytable as a on s.sid=a.sid \
                      inner join collegetable as c on a.cname=c.cname \
                          where c.enrollment < 25000 ").show()

+-----+---+
|sname|gpa|
+-----+---+
|  Amy|3.9|
|  Amy|3.9|
|  Amy|3.9|
|Craig|3.5|
|Craig|3.5|
|Craig|3.5|
|Craig|3.5|
|  Fay|3.8|
|Helen|3.7|
|Irene|3.9|
|Irene|3.9|
|Irene|3.9|
|  Jay|2.9|
|  Jay|2.9|
|  Jay|2.9|
|Craig|3.4|
+-----+---+



- **Q11. Find pair of students (sID) having same GPA. (each pair should occur just once in result)** ?


In [22]:
spark.sql("  select s1.sid as s1, s2.sid as s2 from studenttable as s1 left join studenttable as s2 \
                where s1.sid <> s2.sid and s1.gpa = s2.gpa ").show()

+---+---+
| s1| s2|
+---+---+
|123|654|
|123|876|
|123|456|
|456|654|
|456|876|
|456|123|
|567|765|
|789|543|
|876|654|
|876|456|
|876|123|
|765|567|
|654|876|
|654|456|
|654|123|
|543|789|
+---+---+



- Q12. Find various majors student applied in at college in state MA. 

In [23]:
spark.sql(" select s.sid,s.sname,a.major from studenttable s inner join applytable a \
                on s.sid = a.sid inner join collegetable c on a.cname = c.cname \
                    where c.state = 'MA' ").show()

+---+-----+--------------+
|sid|sname|         major|
+---+-----+--------------+
|345|Craig|bioengineering|
|876|Irene|       biology|
|876|Irene|marine biology|
|543|Craig|            CS|
+---+-----+--------------+



- **Find the college name in which student has not applyed**

- Q1. IDs and names of students who have applied to major in CS at some college.


In [24]:
spark.sql(" select sid,sname from studenttable where sid in \
                (select sid from applytable where major ='CS')  ").show()

+---+-----+
|sid|sname|
+---+-----+
|123|  Amy|
|345|Craig|
|987|Helen|
|876|Irene|
|543|Craig|
+---+-----+



- Q2. Find ID and name of student having same high school size as Jay.


In [25]:
spark.sql(" select sid,sname from studenttable \
                where sizehs in (select sizehs from studenttable where sname in ('Jay')) ").show()

+---+-----+
|sid|sname|
+---+-----+
|234|  Bob|
|765|  Jay|
+---+-----+



- Q3. Find ID and name of student having same high school size as Jay but result should not include Jay.


In [26]:
spark.sql(" select sid,sname from studenttable \
                where sizehs in (select sizehs from studenttable where sname in ('Jay')) \
                    and sname <> 'Jay' ").show()

+---+-----+
|sid|sname|
+---+-----+
|234|  Bob|
+---+-----+



- Q4. Find the name of student with their GPA and Sid whose GPA not equal to GPA of Irene?


In [27]:
spark.sql(" select a.sname, a.gpa from studenttable a \
                where exists (select * from studenttable b where b.sname = 'Irene' and a.gpa <> b.gpa ) ").show()

+------+---+
| sname|gpa|
+------+---+
|   Bob|3.6|
| Craig|3.5|
|Edward|2.9|
|   Fay|3.8|
|  Gary|3.4|
| Helen|3.7|
|   Jay|2.9|
| Craig|3.4|
+------+---+



- Q5. Find college where any student having their name started from J have applied?


In [28]:
spark.sql(" select distinct(cname) from applytable \
                where sid in ( select sid from studenttable where sname like 'J%' ) ").show()

+--------+
|   cname|
+--------+
| Cornell|
|Stanford|
+--------+



- Q6. Find all different major where Irene has applied?


In [29]:
spark.sql("select major from applytable a \
                where exists ( select sid,sname from studenttable s \
                        where s.sid=a.sid and sname ='Irene'  ) ").show()

+--------------+
|         major|
+--------------+
|            CS|
|       biology|
|marine biology|
+--------------+



- Q7. Find IDs of student and major who applied in any of major Irene had applied to?


In [30]:
spark.sql(" select sid,major from applytable \
                where major in ( select major from applytable \
                    where sid in ( select sid from studenttable where sname = 'Irene' ) ) ").show()

+---+--------------+
|sid|         major|
+---+--------------+
|123|            CS|
|123|            CS|
|234|       biology|
|345|            CS|
|987|            CS|
|987|            CS|
|876|            CS|
|876|       biology|
|876|marine biology|
|543|            CS|
+---+--------------+



In [31]:
spark.sql(" select sid,major from applytable a1 \
                where major in ( select major from applytable a2 \
                    where  a1.major=a2.major \
                        and exists ( select sid,sname from studenttable s where sname = 'Irene' and \
                             s.sid = a2.sid ) ) ").show()

+---+--------------+
|sid|         major|
+---+--------------+
|123|            CS|
|123|            CS|
|234|       biology|
|345|            CS|
|987|            CS|
|987|            CS|
|876|            CS|
|876|       biology|
|876|marine biology|
|543|            CS|
+---+--------------+



- Q8. Find IDs of student and major who applied in any of major Irene had applied to? But this time exclude Irene sID from the list.


In [32]:
spark.sql(" select sid,major from applytable \
                where major in ( select major from applytable \
                    where sid in ( select sid from studenttable where sname = 'Irene' ) \
                          ) ").show()

+---+--------------+
|sid|         major|
+---+--------------+
|123|            CS|
|123|            CS|
|234|       biology|
|345|            CS|
|987|            CS|
|987|            CS|
|876|            CS|
|876|       biology|
|876|marine biology|
|543|            CS|
+---+--------------+



- Q9. Give the number of colleges Jay applied to? (Remember count each college once no matter if he applied to same college twice with different major)


In [33]:
spark.sql(" select distinct(cname) from applytable where sid in \
                ( select sid from studenttable where sname = 'Jay' ) ").show()

+--------+
|   cname|
+--------+
| Cornell|
|Stanford|
+--------+



- Q10. Find sID of student who applied to more or same number of college where Jay has applied?


In [46]:
spark.sql(" select s.sid,count(distinct a.cname) as count_c \
                from studenttable s inner join applytable a on s.sid=a.sid \
                    group by s.sid  having count_c >= \
                        ( select count( distinct cname ) from applytable \
                            where sid in ( select sid from studenttable where sname = 'Jay' ) ) ").show()


+---+-------+
|sid|count_c|
+---+-------+
|876|      2|
|123|      3|
|345|      2|
|765|      2|
|987|      2|
+---+-------+



- Q11. Find details of Students who applied to major CS but not applied to major EE? (sID 987, 876, 543 should only be include in result)


In [71]:
spark.sql(" select sid,sname from studenttable \
                where sid in ( select sid from applytable where major = 'CS'and \
                    sid not in ( select sid from applytable where major = 'EE' ) ) ").show()

+---+-----+
|sid|sname|
+---+-----+
|987|Helen|
|876|Irene|
|543|Craig|
+---+-----+



- Q12. All colleges such that some other college is in same state. (Cornell should not be part of result as no other college in New York Hint: use exists)


In [ ]:
spark.sql(" select  ").show()

- Q13. Find the college with highest enrollment.


In [76]:
spark.sql(" select cname from collegetable where enrollment = (select max(enrollment) from collegetable) ").show()

+-------+
|  cname|
+-------+
|Harvard|
+-------+



- Q14. Find name of student having lowest GPA.

In [ ]:
spark.sql("  ").show()

- Q15. Find the most popular major.


In [ ]:
spark.sql("  ").show()

- Q16. Find sID, sName, sizeHS of all students NOT from smallest HS

In [ ]:
spark.sql("  ").show()

- Q17. Find the name of student who applies to all the colleges where sID 987 has applied?

In [ ]:
spark.sql("  ").show()